<a href="https://colab.research.google.com/github/arthurbabey/road66/blob/arthur/cnn_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

TensorFlow 2.x selected.
Found GPU at: /device:GPU:0


In [0]:
%matplotlib inline
import matplotlib.image as mpimg
import numpy as np
import matplotlib.pyplot as plt
import os,sys
from PIL import Image
import sys
sys.path.append('/Users/arthurbabey/Documents/master2/ML/road66/scripts')
import keras 



from keras import regularizers
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten

In [0]:

def load_image(infilename):
    data = mpimg.imread(infilename)
    return data

def img_float_to_uint8(img):
    rimg = img - np.min(img)
    rimg = (rimg / np.max(rimg) * 255).round().astype(np.uint8)
    return rimg

# Concatenate an image and its groundtruth
def concatenate_images(img, gt_img):
    nChannels = len(gt_img.shape)
    w = gt_img.shape[0]
    h = gt_img.shape[1]
    if nChannels == 3:
        cimg = np.concatenate((img, gt_img), axis=1)
    else:
        gt_img_3c = np.zeros((w, h, 3), dtype=np.uint8)
        gt_img8 = img_float_to_uint8(gt_img)
        gt_img_3c[:,:,0] = gt_img8
        gt_img_3c[:,:,1] = gt_img8
        gt_img_3c[:,:,2] = gt_img8
        img8 = img_float_to_uint8(img)
        cimg = np.concatenate((img8, gt_img_3c), axis=1)
    return cimg

def img_crop(im, w, h):
    list_patches = []
    imgwidth = im.shape[0]
    imgheight = im.shape[1]
    is_2d = len(im.shape) < 3
    for i in range(0,imgheight,h):
        for j in range(0,imgwidth,w):
            if is_2d:
                im_patch = im[j:j+w, i:i+h]
            else:
                im_patch = im[j:j+w, i:i+h, :]
            list_patches.append(im_patch)
    return list_patches


# Convert array of labels to an image

def label_to_img(imgwidth, imgheight, w, h, labels):
    im = np.zeros([imgwidth, imgheight])
    idx = 0
    for i in range(0,imgheight,h):
        for j in range(0,imgwidth,w):
            im[j:j+w, i:i+h] = labels[idx]
            idx = idx + 1
    return im






def value_to_class(v, foreground_threshold=0.25): #modifier leur fonction en ajoutant f_t en param

    df = np.sum(v)
    if df > foreground_threshold:
        return 1
    else:
        return 0






In [0]:
def rotate_images(X, Y , degrees):
    """
    increase the number of data
    by adding rotations of the base data
    """

    X = np.array(X)
    Y = np.array(Y)
    rotimg = np.zeros(X.shape)
    rotgtimg = np.zeros(Y.shape)
    
    Xtemp = X
    Ytemp = Y
    
    #rotate all images by degree and add them to the data vector
    for degree in degrees:
        for i in range(len(Xtemp)):
            rotimg[i] = rotate(Xtemp[i], degree, resize=False, mode='reflect')
            rotgtimg[i] = rotate(Ytemp[i], degree, resize=False, mode='reflect')
        X = np.concatenate([X,rotimg])
        Y = np.concatenate([Y,rotgtimg])

    return X,Y



In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# Loaded a set of images
root_dir = "drive/My Drive/training/"

image_dir = root_dir + "images/"
files = os.listdir(image_dir)
n = len(files) # Use all images
print("Loading " + str(n) + " testing images")
imgs = [load_image(image_dir + files[i]) for i in range(n)]

gt_dir = root_dir + "groundtruth/"
print("Loading " + str(n) + " groundtruth")
gt_imgs = [load_image(gt_dir + files[i]) for i in range(n)]

Loading 100 testing images
Loading 100 groundtruth


In [0]:
from skimage.transform import rotate

imgs, gt_imgs = rotate_images(imgs, gt_imgs, [15, 45])

In [0]:
# 
# Preparation training data
# 


# Extract patches from input images
patch_size = 16 # each patch is 16*16 pixels

n = len(imgs)

img_patches = [img_crop(imgs[i], patch_size, patch_size) for i in range(n)]
gt_patches = [img_crop(gt_imgs[i], patch_size, patch_size) for i in range(n)]

# Linearize list of patches
img_patches = np.asarray([img_patches[i][j] for i in range(len(img_patches)) for j in range(len(img_patches[i]))])
gt_patches =  np.asarray([gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))])

img_patches = np.asarray(img_patches)
gt_patches = np.asarray(gt_patches)

x_train = img_patches
y_train = np.asarray([value_to_class(np.mean(gt_patches[i])) for i in range(gt_patches.shape[0])])

print("62500 patches of 16x16 pixels for training are stored in x_train")
print("62500 binary values (1 = road, 0 = bckgrnd) are stored in y_train")

62500 patches of 16x16 pixels for training are stored in x_train
62500 binary values (1 = road, 0 = bckgrnd) are stored in y_train


In [0]:
# 
# Preparation testing data to make a submission
# 

# 
# Ordering image
# 
root_testdir = "drive/My Drive/test_set_images"
test_names = os.listdir(root_testdir)

num_test = len(test_names)
order = [int(test_names[i].split("_")[1]) for i in range(num_test)]
index = np.argsort(order)

# Load image and reorder them
imgs_test = [load_image(os.path.join(root_testdir, test_names[i], test_names[i]) + ".png") 
             for i in range(num_test)]
imgs_test = [imgs_test[i] for i in index]

# Crop images in patch and Linearize list of patches
patch_size = 16 # each patch is 16*16 pixels

img_patches_test = [img_crop(imgs_test[i], patch_size, patch_size) for i in range(num_test)]


# Linearize list of patches
y_test = np.asarray([img_patches_test[i][j] for i in range(len(img_patches_test)) 
                     for j in range(len(img_patches_test[i]))])

print("72200 patches of 16x16 pixels from the test set are stored in y_test")

72200 patches of 16x16 pixels from the test set are stored in y_test


In [0]:
y_train = np.array(y_train, ndmin = 2)            
y_train = keras.utils.to_categorical(y_train.T, 2)

In [0]:

c0 = 0  # bgrd
c1 = 0  # road
for i in range(len(y_train)):
    if y_train[i][0] == 1:
        c0 = c0 + 1
    else:
        c1 = c1 + 1
print('Number of data points per class: c0 = ' + str(c0) + ' c1 = ' + str(c1))

print('Balancing training data...')
min_c = min(c0, c1)
idx0 = [i for i, j in enumerate(y_train) if j[0] == 1]
idx1 = [i for i, j in enumerate(y_train) if j[1] == 1]
new_indices = idx0[0:min_c] + idx1[0:min_c]
print(len(new_indices))
print(x_train.shape)
x_train = x_train[new_indices, :, :, :]
y_train = y_train[new_indices]

train_size = y_train.shape[0]

c0 = 0
c1 = 0
for i in range(len(y_train)):
    if y_train[i][0] == 1:
        c0 = c0 + 1
    else:
        c1 = c1 + 1
print('Number of data points per class: c0 = ' + str(c0) + ' c1 = ' + str(c1))

Number of data points per class: c0 = 139885 c1 = 47615
Balancing training data...
95230
(187500, 16, 16, 3)
Number of data points per class: c0 = 47615 c1 = 47615


In [0]:
batch_size = 32
num_classes = 2
epochs = 30
#data_augmentation = True


In [0]:
from keras.callbacks import Callback,ModelCheckpoint
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasClassifier
import keras.backend as K

In [0]:
model = tf.keras.models.Sequential() #create a model

model.add(tf.keras.layers.Conv2D(16, (5, 5), activation = tf.nn.relu , padding='same', #first conv + relu layer
                 data_format='channels_last',
                 input_shape=x_train.shape[1:]))
model.add(tf.keras.layers.Conv2D(16, (5, 5), activation = tf.nn.relu , padding='same', #first conv + relu layer
                 data_format='channels_last',
                 input_shape=x_train.shape[1:]))

model.add(tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=2, padding='same', data_format=None)) #first pooling layer

model.add(tf.keras.layers.Conv2D(32, (5, 5), activation = tf.nn.relu , padding='same')) #conv + relu layer
model.add(tf.keras.layers.Conv2D(32, (5, 5), activation = tf.nn.relu , padding='same')) #conv + relu layer

model.add(tf.keras.layers.MaxPooling2D(pool_size=(3, 3), strides=2, padding='same', data_format=None)) #pooling layer


model.add(tf.keras.layers.Flatten()) #flatt layer before feeding the fully connected layer
model.add(tf.keras.layers.Dense(128, activation = tf.nn.relu, activity_regularizer=regularizers.l2()))
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Dense(2, activation = tf.nn.softmax)) #.Fully Connected layer with 2 outputs


In [0]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(128, activation = tf.nn.relu))
model.add(tf.keras.layers.Dense(10, activation = tf.nn.softmax))

In [0]:
model.compile(loss='binary_crossentropy', #compile the model
              optimizer='adam',
              metrics=['categorical_accuracy'])


model.fit(x_train, y_train,              #fitting the model 
              batch_size=batch_size,
              epochs=epochs,
              #validation_data=(x_test, y_test),
              #callbacks = [tensorboard_callback]
         )

Train on 95230 samples
Epoch 1/30
95230/95230 [==============================] - 14s 151us/sample - loss: 0.4760 - categorical_accuracy: 0.7797
Epoch 2/30
95230/95230 [==============================] - 13s 140us/sample - loss: 0.3875 - categorical_accuracy: 0.8287
Epoch 3/30
95230/95230 [==============================] - 13s 138us/sample - loss: 0.3627 - categorical_accuracy: 0.8427
Epoch 4/30
95230/95230 [==============================] - 13s 138us/sample - loss: 0.3465 - categorical_accuracy: 0.8481
Epoch 5/30
95230/95230 [==============================] - 13s 138us/sample - loss: 0.3375 - categorical_accuracy: 0.8524
Epoch 6/30
95230/95230 [==============================] - 13s 139us/sample - loss: 0.3315 - categorical_accuracy: 0.8561
Epoch 7/30
95230/95230 [==============================] - 13s 139us/sample - loss: 0.3260 - categorical_accuracy: 0.8593
Epoch 8/30
95230/95230 [==============================] - 13s 138us/sample - loss: 0.3205 - categorical_accuracy: 0.8624
Epoch 9/3

In [0]:
predictions = model.predict(y_test, batch_size=256, verbose=1, 
                            )

72200/1 [===============================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [0]:
pred = np.zeros(predictions.shape[0]) 
pred[predictions[:,0]< predictions[:,1]] = 1 #merging predictions to 1d vector with value 0 or 1

In [0]:
def create_submission(y_pred, submission_filename, patch_size = 16, images_size = 608):
    n_patches = images_size // patch_size
    y_pred = np.reshape(y_pred, (-1, n_patches, n_patches))

    with open(submission_filename, 'w') as f:
        f.write('id,prediction\n')
        for i in range(y_pred.shape[0]):
            img = y_pred[i]
            for j in range(img.shape[0]):
                for k in range(img.shape[1]):
                    name = '{:03d}_{}_{},{}'.format(i + 1, j * patch_size, k * patch_size, int(img[j,k]))
                    f.write(name + '\n')

In [0]:
create_submission(pred, "cnn4433_submission.csv")
